# Detecting blinking using a videostream with dlib and OpenCV

After making the code for detecting open and closed eyes through images, I would like to use a video and detect the time each blink.

The code detect the blinks, but also calulates the time between each blink. If the person has not blinked for more than 10 seconds, the "please blink" message gets printed.

In [ ]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance
import time

#Download the shape predictor file
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

#Load the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Download from dlib repo

--2025-02-10 08:12:54--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-02-10 08:12:54--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  17.9MB/s    in 4.4s    

2025-02-10 08:13:00 (13.9 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
#Eye aspect ratio (EAR) calculation
def eye_aspect_ratio(eye_points):
    A = distance.euclidean(eye_points[1], eye_points[5])
    B = distance.euclidean(eye_points[2], eye_points[4])
    C = distance.euclidean(eye_points[0], eye_points[3])
    ear = (A + B) / (2.0 * C)
    return ear

#Thresholds
EAR_THRESHOLD = 0.25  #Below this, eyes are considered closed
BLINK_REMINDER_THRESHOLD = 10  #Alert if no blink for 10 seconds

#Video capture
cap = cv2.VideoCapture('/content/20250208_111726.mp4')
output_video_path = "/content/output_with_blinks.mp4"

#Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(f"FPS: {fps}")

#Define codec and initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*"avc1") #or mp4v or avc1
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

#Blink tracking variables
blink_count = 0
blink_start_time = None
blink_detected = False
blink_alert_shown = False
show_warning = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    #Rotate the frame for proper orientation (If the video is vertical)
    #frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

    #Get the current frame timestamp in seconds
    frame_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

    #If working with webcam then replace frame time with current time
    #current_time = time.time()

    #Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        #Extract eye landmarks
        left_eye_pts = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)]
        right_eye_pts = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)]

        #Compute EAR for both eyes
        left_ear = eye_aspect_ratio(left_eye_pts)
        right_ear = eye_aspect_ratio(right_eye_pts)
        avg_ear = (left_ear + right_ear) / 2.0

        #Detect blink
        if avg_ear < EAR_THRESHOLD:
            if not blink_detected:  #New blink detected
                blink_count += 1
                blink_detected = True
                show_warning = False  #reset warning when blink detected

                #Calculate time since last blink
                if blink_start_time is not None:
                    time_between_blinks = frame_time - blink_start_time
                    print(f"Blink #{blink_count}: {time_between_blinks:.2f} seconds since last blink")
                else:
                    print(f"Blink #{blink_count}: First blink detected!")

                blink_start_time = frame_time
                blink_alert_shown = False  #Reset alert flag
        else:
            blink_detected = False  #Reset when eyes reopen

    #Check if more than 10 seconds have passed since last blink
    if blink_start_time and (frame_time - blink_start_time) >= BLINK_REMINDER_THRESHOLD:
        if not blink_alert_shown:  #Show only once per reminder
            print("⚠️ BLINK PLEASE! You haven't blinked in the past 10+ seconds! ⚠️")
            blink_alert_shown = True
            show_warning = True  #Activate warning display

    #Display blink count on video
    cv2.putText(frame, f"Blinks: {blink_count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    #Show warning message only when needed
    if show_warning:
        cv2.putText(frame, " BLINK! ", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)

    #Rotate back before saving (for proper playback, if video is vertical)
    #frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

    #Write the frame to output video
    out.write(frame)

    #Show the video feed for debugging
    #cv2_imshow(frame)


#cap.release()
#out.release()

print(f"Video saved as {output_video_path}")

# Detecting open or closed eyes from images using dlib

In [ ]:
def eye_aspect_ratio(eye_points):
    """Calculate the Eye Aspect Ratio (EAR) to detect if an eye is open or closed"""
    A = distance.euclidean(eye_points[1], eye_points[5])
    B = distance.euclidean(eye_points[2], eye_points[4])
    C = distance.euclidean(eye_points[0], eye_points[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Threshold to classify eye state
EAR_THRESHOLD = 0.25

# Load image
image = cv2.imread("/content/20250128_113657.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = detector(gray)

for face in faces:
    landmarks = predictor(gray, face)

    # Extract eye landmarks (dlib indices)
    left_eye_pts = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)]
    right_eye_pts = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)]

    # Compute EAR for both eyes
    left_ear = eye_aspect_ratio(left_eye_pts)
    right_ear = eye_aspect_ratio(right_eye_pts)

    # Average EAR
    avg_ear = (left_ear + right_ear) / 2.0

    # Determine eye state
    if avg_ear < EAR_THRESHOLD:
        state = "Closed"
    else:
        state = "Open"

    print(f"Eye state: {state}")

# Show the image
#cv2_imshow(image)